In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)

# TSP

These "problem_g_X.npy" instances are **symmetric metric Traveling Salesman Problems** on a complete undirected graph.

Symmetric metric TSP properties:
* *No negative entries* and *zero diagonal* - Travel costs can't be negative, and staying in the same city is free.
* *Symmetric matrix* - The cost from A to B is the same as the cost from B to A.
* *Triangle inequality* - Going directly from A to B is never more expensive than going through a third city C.

---

## Implementation Details

Based on well-established literature, we decided to build the algorithm as a **Genetic Algorithm (GA)** hybridized with a **2-OPT** local search, making it a **memetic algorithm** tailored for the symmetric metric TSP.  
We tried to ensure an effective balance between **exploration** (via crossover and mutation) and **exploitation** (via local search).

Below you can find a detailed description of each algorithmic component.

### 1. Fitness Evaluation

**Function:** `fitness(tour, dist_matrix)`  
This function computes the total cost of a tour by summing up the distances between consecutive cities, including the return to the starting city.


In [2]:
def fitness(tour: np.ndarray, dist_matrix: np.ndarray) -> float:
    """Compute the total (closed) tour length for a given sequence of cities (TSP)."""
    return float(np.sum(dist_matrix[tour, np.roll(tour, -1)]))


### 2. Initialization: Nearest Neighbor (NN)

**Functions:**  
- `nearest_neighbor_tour(start, dist_matrix)`  
- `init_population_nn(pop_size, dist_matrix)`

The initialization uses the **Nearest Neighbor (NN)** heuristic, which constructs a tour starting from each city and repeatedly visiting the nearest unvisited city.  
This produces **feasible** and **low-cost initial solutions**, leveraging the triangle inequality to guarantee reasonably short paths.

To increase diversity, after generating tours from all possible starting cities, additional tours are produced by **randomly perturbing** existing NN tours through small swaps.

In [3]:
def nearest_neighbor_tour(start: int, dist_matrix: np.ndarray) -> np.ndarray:
    """Construct a tour using the nearest neighbor heuristic starting from 'start'."""
    n_cities = dist_matrix.shape[0]
    unvisited = set(range(n_cities))
    tour = [start]
    unvisited.remove(start)
    current = start

    while unvisited:
        closest_city = min(unvisited, key=lambda j: dist_matrix[current, j])
        tour.append(closest_city)
        unvisited.remove(closest_city)
        current = closest_city

    return np.array(tour, dtype=int)

def init_population_nn(pop_size: int, dist_matrix: np.ndarray) -> list[np.ndarray]:
    """Initialize population using NN heuristic from different starting points."""
    n_cities = dist_matrix.shape[0]
    population = []
    starts = list(range(n_cities))
    i = 0

    while len(population) < pop_size:
        if i < n_cities:
            start = starts[i]
            base = nearest_neighbor_tour(start, dist_matrix)
            population.append(base)
        else:
            b = nearest_neighbor_tour(random.choice(starts), dist_matrix).copy()
            for _ in range(random.randint(1, 3)):
                i, j = random.sample(range(n_cities), 2)
                b[i], b[j] = b[j], b[i]
            population.append(b)
        i += 1

    return population[:pop_size]

### 3. Crossover Operator: Order Crossover (OX)

**Function:** `order_crossover(parent1, parent2)`

The **Order Crossover (OX)** operator generates two offspring by preserving the **relative order** of cities from one parent while filling remaining positions according to the other.  
This operator is well suited for permutation problems like TSP, where each city must appear exactly once.

In [4]:
def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Order Crossover (OX): creates two offspring preserving relative order"""
    n_cities = len(parent1)
    a, b = sorted(random.sample(range(n_cities), 2))

    child1 = np.full(n_cities, -1, dtype=int)
    child2 = np.full(n_cities, -1, dtype=int)

    child1[a:b+1] = parent1[a:b+1]
    child2[a:b+1] = parent2[a:b+1]

    def fill(child, donor):
        pos = (b + 1) % n_cities
        donor_genes = np.concatenate((donor[b + 1 :], donor[: b + 1]))
        for gene in donor_genes:
            if gene not in child:
                child[pos] = gene
                pos = (pos + 1) % n_cities
        return child

    return fill(child1, parent2), fill(child2, parent1)

### 4. Mutation Operator: Swap Mutation

**Function:** `swap_mutation(ind, n_swaps=1)`

This mutation randomly selects pairs of cities in a tour and **swaps** them.  
The operator introduces local perturbations that help the GA escape premature convergence and explore nearby solutions.

In [5]:
def swap_mutation(ind: np.ndarray, n_swaps: int = 1) -> np.ndarray:
    """Swap mutation: exchanges n_swaps random pairs"""
    new_ind = ind.copy()
    n_cities = len(ind)
    for _ in range(n_swaps):
        i, j = random.sample(range(n_cities), 2)
        new_ind[i], new_ind[j] = new_ind[j], new_ind[i]
    return new_ind

### 5. Local Search: 2-OPT Heuristic

**Function:** `two_opt_tsp(tour, dist_matrix)`

It iteratively checks every pair of edges `(a,b)` and `(c,d)` in the tour and evaluates whether replacing them with `(a,c)` and `(b,d)` reduces the total length.

If the new configuration shortens the tour then the subsequence between `b` and `c` is **reversed**.

Because the TSP is metric (triangle inequality holds) and symmetric, uncrossing edges always leads to valid and potentially shorter tours.  
Thus, 2-OPT efficiently removes crossings and finds high-quality local minima.

In [6]:
def two_opt_tsp(tour: np.ndarray, dist_matrix: np.ndarray) -> np.ndarray:
    """2-opt for symmetric TSP"""
    best = tour.copy()
    n = len(best)
    improved = True

    while improved:
        improved = False
        for i in range(n - 1):
            for j in range(i + 2, n):
                if i == 0 and j == n - 1:
                    # avoid breaking the same edge
                    continue
                a, b = best[i], best[(i + 1) % n]
                c, d = best[j], best[(j + 1) % n]
                delta = (dist_matrix[a, b] + dist_matrix[c, d]) - (
                    dist_matrix[a, c] + dist_matrix[b, d]
                )
                if delta > 1e-12:
                    best[i + 1 : j + 1] = np.flip(best[i + 1 : j + 1])
                    improved = True
    return best



### 6. Selection Strategy: Tournament Selection with Elitism

**Function:** `tournament_selection(population, fitnesses, k)`

Selection is based on **k-tournament selection**, where `k` individuals are chosen at random and the best among them (lowest cost) is cloned for reproduction.

**Elitism:**  
A small number of top-performing individuals are copied unchanged into the next generation.  
This guarantees that the best solutions are never lost due to stochastic variation and steadily improves the global best.

**Balance:**  
Tournament selection maintains selective pressure toward good solutions, while elitism ensures **steady convergence** without loss of the current best.

In [7]:
def tournament_selection(population: list[np.ndarray], fitnesses: list[float], k: int) -> np.ndarray:
    """Tournament selection of size k"""
    selected_idx = random.sample(range(len(population)), k)
    best = min(selected_idx, key=lambda i: fitnesses[i])
    return population[best].copy()


### 7. Main Evolutionary Loop

**Function:** `symmetric_TSP(...)`

The core of the algorithm integrates all the above components into a full **memetic evolutionary framework**:

1. **Initialization:** Generate a population using NN tours + perturbations.  
2. **Evaluation:** Compute fitness for each tour using `tour_length`.  
3. **Selection:** Preserve best individuals for the next generation.  
4. **Crossover:** With probability `crossover_rate`, apply OX to generate new offspring.  
5. **Mutation:** With probability `mutation_rate`, apply swap mutation.  
6. **Local Search (2-OPT):** With probability `two_opt_rate`, refine offspring via 2-OPT.  
7. **Replacement:** Form the new population and repeat for `generations` iterations.

In [8]:
def symmetric_TSP(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3,
    two_opt_rate: float = 0.1
) -> tuple[np.ndarray, float]:
    """
    EA for symmetric TSP
    with NN initialization and operators:
      - tournament selection
      - OX crossover
      - swap mutation
      - 2-opt local search
      - elitism
    """

    population = init_population_nn(pop_size, dist_matrix)
    fitnesses = [fitness(ind, dist_matrix) for ind in population]

    best_overall = None
    best_overall_fit = float("inf")

    for gen in range(1, generations + 1):
        # sort population by fitness minimization
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        # elitism
        new_population = []
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # generation of offsprings
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)

            while np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            # OX crossover
            if random.random() < crossover_rate:
                child1, child2 = order_crossover(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # mutation
            if random.random() < mutation_rate:
                child1 = swap_mutation(child1)
            if random.random() < mutation_rate:
                child2 = swap_mutation(child2)

            # 2-opt local search
            if random.random() < two_opt_rate:
                child1 = two_opt_tsp(child1, dist_matrix)
            if random.random() < two_opt_rate:
                child2 = two_opt_tsp(child2, dist_matrix)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        # updation
        population = new_population.copy()
        fitnesses = [fitness(ind, dist_matrix) for ind in population]

        # progress
        if gen % 10 == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit  # type: ignore[return-value]


## Why These Components Work Well Together

The algorithm achieves strong results on **symmetric metric TSPs** because of the following synergies:

1. **Nearest Neighbor initialization** exploits the triangle inequality to generate short tours quickly.  
2. **Order Crossover (OX)** respects partial order, preserving city adjacency useful under symmetric distances.  
3. **Swap mutation** introduces small, feasible changes without large disruptions.  
4. **2-OPT local search** leverages the triangle inequality to safely eliminate crossings and improve tours.  
5. **Tournament + Elitism** ensures a steady convergence toward global minima without losing high-quality solutions.

## Experiments

### G 10

In [ ]:
# Load data
dist_matrix = np.load('/problem_g_10.npy')

In [ ]:
POP_SIZE = 50
GENERATIONS = 100
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 2
TWO_OPT_RATE = 1

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 1497.6636
Gen   10 | Best: 1497.6636
Gen   20 | Best: 1497.6636
Gen   30 | Best: 1497.6636
Gen   40 | Best: 1497.6636
Gen   50 | Best: 1497.6636
Gen   60 | Best: 1497.6636
Gen   70 | Best: 1497.6636
Gen   80 | Best: 1497.6636
Gen   90 | Best: 1497.6636
Gen  100 | Best: 1497.6636
Best cost: 1497.6636482252907
Best tour: [3 2 8 0 7 9 5 4 6 1]


### G 20

In [ ]:
# Load data
dist_matrix = np.load('/problem_g_20.npy')

In [ ]:
POP_SIZE = 50
GENERATIONS = 100
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 2
TWO_OPT_RATE = 1

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 1756.7636
Gen   10 | Best: 1755.5147
Gen   20 | Best: 1755.5147
Gen   30 | Best: 1755.5147
Gen   40 | Best: 1755.5147
Gen   50 | Best: 1755.5147
Gen   60 | Best: 1755.5147
Gen   70 | Best: 1755.5147
Gen   80 | Best: 1755.5147
Gen   90 | Best: 1755.5147
Gen  100 | Best: 1755.5147
Best cost: 1755.5146770830047
Best tour: [17  4 10 18  5  0 11 13  6  8  3 15 19 14 16  1  7 12  9  2]


### G 50

In [ ]:
# Load data
dist_matrix = np.load('/problem_g_50.npy')

In [ ]:
POP_SIZE = 50
GENERATIONS = 100
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 2
TWO_OPT_RATE = 1
SEED = 1234

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 2968.3612
Gen   10 | Best: 2629.9867
Gen   20 | Best: 2629.9867
Gen   30 | Best: 2629.9867
Gen   40 | Best: 2629.9867
Gen   50 | Best: 2629.9867
Gen   60 | Best: 2629.9867
Gen   70 | Best: 2629.9867
Gen   80 | Best: 2629.9867
Gen   90 | Best: 2629.9867
Gen  100 | Best: 2629.9867
Best cost: 2629.986685707395
Best tour: [ 8 38 49 48 40 25 27  0 14 47 46 18 20 31 24 42 32 29 19  9  2  4 43 22
 11  3 10 34  7  6 44 45 23 30  5 16 35 13 15 28 12  1 41 36 33 21 37 39
 26 17]


### G 100

In [ ]:
# Load data
dist_matrix = np.load('/problem_g_100.npy')

In [ ]:
POP_SIZE = 50
GENERATIONS = 100
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 2
TWO_OPT_RATE = 1

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 4490.9469
Gen   10 | Best: 3974.4793
Gen   20 | Best: 3966.4118
Gen   30 | Best: 3961.9822
Gen   40 | Best: 3954.2328
Gen   50 | Best: 3952.5121
Gen   60 | Best: 3952.5121
Gen   70 | Best: 3952.5121
Gen   80 | Best: 3952.5121
Gen   90 | Best: 3952.5121
Gen  100 | Best: 3952.5121
Best cost: 3952.5121039390847
Best tour: [66 73 45 79 94 44 17 35 38 95 62 29 61 23  9 82 59 65 39 98 96 76 13 26
 25 80 19 40 24 50 83 89 85 77 15 91  6 12 78 88 92 43 87 86 55 75 34  4
 93 46 36  8  1 69 22  5 49  7 99 63 47 71 27 20 74 41 14  0 33 54 67 37
 32 10 48 72 31 42 64 97 30 81 68 52 90 16 53 70 56 18 57 58  2 11 60 21
 28 51  3 84]


### G 200

In [ ]:
# Load data
dist_matrix = np.load('/problem_g_200.npy')

In [ ]:
POP_SIZE = 50
GENERATIONS = 100
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 2
TWO_OPT_RATE = 1
SEED = 1234

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 6264.1471
Gen   10 | Best: 5564.9904
Gen   20 | Best: 5550.5269
Gen   30 | Best: 5500.4049
Gen   40 | Best: 5491.4883
Gen   50 | Best: 5489.8888
Gen   60 | Best: 5485.7684
Gen   70 | Best: 5485.7684
Gen   80 | Best: 5484.0585
Gen   90 | Best: 5482.0231
Gen  100 | Best: 5482.0231
Best cost: 5482.023091792661
Best tour: [174  26  28  63  79 188  90  20 198  23 109 163 134 179  14 135  96  95
 177   4  32  70 149  58 105  15  72 189 161 113  36  44 120 187 168  73
  92 151  25  87 121 141  89 182 145 166  29  91  69 190 147   5 103  54
  31  68  93  18 104 130  13  39 158  22 101 118  47  50   2 175 119   9
  38  43  46   6 136  24 178 138 193 172 125 112 128 106  66  99  76 124
 181 185  51  97  40 171  41 117  67 186 167  98 154 160 148  21  80 157
  82 196 184 126 116 123  85 199 173  48 107 115 144  86  59 197  62  42
 100 140  71  74 131  57 132  77  56  10  52 156 146 143   1 111  78 176
  17 133 155  53 102 170 142   7  27 165  64  19 110  33  81  88 122  34
   3  

### G 500

In [ ]:
# Load data
dist_matrix = np.load('/problem_g_500.npy')

In [ ]:
POP_SIZE = 200
GENERATIONS = 100
CROSSOVER_RATE = 0.99
MUTATION_RATE = 0.3
ELITISM = 5
TOURN_K = 5
TWO_OPT_RATE = 0.1

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 9654.6236
Gen   10 | Best: 8733.5616
Gen   20 | Best: 8646.8386
Gen   30 | Best: 8599.0612
Gen   40 | Best: 8559.1527
Gen   50 | Best: 8535.3149
Gen   60 | Best: 8528.1303
Gen   70 | Best: 8498.7931
Gen   80 | Best: 8482.3294
Gen   90 | Best: 8459.4869
Gen  100 | Best: 8457.5443
Best cost: 8457.544296924623
Best tour: [129 350 285 123 477  55 374 222  78 472 365  53  28  38  79 133 339 418
 393 383 395  85 351 230 328 163 438 380  90 176  22 225 154 210 265  57
 228 396 323 327 321 337 254 246 201 425 363 126 238 308  13 300  52 205
 302  15 180 240 282 455  18 232 279   9  81 259 370 311 407 107  54 368
  58 409 464  76 317 309 237  51  83 245 389 280  34  16 216 263 448 289
  11 399 465 442 434 229 392 156 378  19  92 373 483 191  45  29 261  68
 457 498 158 165  44 145 244 190 177 312 456   0 108 352  56 495 333 253
 340  62  98 481 491 277 172 202 274 199 458 482 171 155  12 446 416 314
 184 369  26 248 249 305 379 330 268 342 288 168 132 146 386  67   5   6
 130 4

### G 1000

In [ ]:
# Load data
dist_matrix = np.load('/problem_g_1000.npy')

In [ ]:
POP_SIZE = 5
GENERATIONS = 100
CROSSOVER_RATE = 0.99
MUTATION_RATE = 0.3
ELITISM = 1
TOURN_K = 1
TWO_OPT_RATE = 1

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 14202.4585
Gen   10 | Best: 12510.1621
Gen   20 | Best: 12421.2745
Gen   30 | Best: 12419.4749
Gen   40 | Best: 12341.1464
Gen   50 | Best: 12280.0274
Gen   60 | Best: 12280.0274
Gen   70 | Best: 12255.5639
Gen   80 | Best: 12255.5639
Gen   90 | Best: 12246.3337
Gen  100 | Best: 12246.3337
Best cost: 12246.33373728384
Best tour: [496 836 384 906 552 651 807 246  94 656 696 136  51 759 143 785 500 206
 810 638 809 368 198 977 178 418  47 109 557 149 998 997 801 460 844 562
 242 783 567  66 389 140 194 440 439 616 252  45 697 340 597 192 427 890
 250 834 390  71 593 186 762 881 853 429 459  75 376 507 631 961 541 657
 619 770 699 105 617  17 894 465 258 419  29 940 999 870  76 839 469 670
 404 808 539 293 462 195 750 624 401 632 528 328 286 422 385 338 972 904
 967 941 523 450 363 645  24 117 415 548 587 342 411   4 150 620 487 478
 937 603 882 571  67  98 268 475 568 324 971 346 897 851 457 280 326 861
 982 784 732 927 708 274 257  86 652 821 159 614 240 234 679 181 277

# Non-metric ATSP

These "problem_r1_N.npy" instances represent **non-metric Asymmetric Traveling Salesman Problems (ATSP)** on a **complete directed graph**.

Non-metric ASTP properties:
* *No negative entries* and *zero diagonal* - Travel costs can't be negative, and staying in the same city is free.
* *Asymmetry* - The travel cost from A -> B differs from the cost from B -> A.  
* *Zero diagonal* - No cost is incurred when remaining in the same city, consistent with TSP conventions.  
* *Triangle inequality violated* - Direct routes are not guaranteed to be cheaper than indirect ones.

Because of the properties of ATSP, many assumptions that hold for the metric symmetric TSP, such as safe detour elimination or consistent edge reversals, no longer apply.  
This requires structural changes in both initialization and crossover design.

---

## Implementation Details

To handle this non-metric, asymmetric scenario, the Genetic Algorithm (GA) is redesigned to correctly process directional costs and non-metric relationships.  
Components from the symmetric metric version, such as Nearest Neighbor (NN) initialization, 2-OPT and Order Crossover (OX), were replaced or removed, as they rely on symmetry and the triangle inequality.

The design choices in this implementation are informed by the analysis presented in  
[*Appropriate Combination of Crossover Operator and Local Search for TSP & Variants*](https://www.sciencedirect.com/org/science/article/pii/S1546221824002674?).

Below is a detailed breakdown of the updated algorithmic structure and its differences from the metric TSP version.
Only the differences are discussed.

### 1. Initialization: Random Population

**Function:** `init_population_random(pop_size, dist_matrix)`

The initialization step now creates a **fully random population** of tours (random permutations of city indices).  
This replaces the **Nearest Neighbor (NN)** initialization used in the symmetric version.
The NN heuristic assumes that the closest unvisited city always leads toward a shorter overall route, an assumption that depends on the triangle inequality.

In [9]:
def init_population_random(pop_size: int, dist_matrix: np.ndarray) -> list[np.ndarray]:
    """Create a random initial population of tours"""
    random.seed(random_seed)
    np.random.seed(random_seed)
    n_cities = dist_matrix.shape[0]
    # Each individual is a random permutation of city indices.
    return [np.random.permutation(n_cities) for _ in range(pop_size)]


### 2. Crossover Operator: Sequential Constructive Crossover (SCX)

**Function:** `scx_crossover(parent1, parent2, dist_matrix)`

The **Sequential Constructive Crossover (SCX)** is implemented to replace Order Crossover (OX).  
Unlike OX, which only preserves the order of cities, SCX constructs offspring step by step, guided by actual **arc costs**.  
At each step, it selects the next city that results in the smallest cost from the current one, using edge information from both parents.

It is particularly suitable for the ATSP because it takes into account the directions and it constructs each offspring sequentially while always ensuring solution feasibility.


In [10]:
def scx_crossover(parent1: np.ndarray, parent2: np.ndarray, dist_matrix: np.ndarray) -> np.ndarray:
    """
    SCX (Sequential Constructive Crossover): builds a child by choosing the next city based on direct arc cost
    """
    n_cities = len(parent1)
    child = [parent1[0]] # first city of parent 1
    remaining = set(range(n_cities))
    remaining.remove(parent1[0])

    while remaining:
        current = child[-1]

        # next city positions in each parent
        idx1 = np.where(parent1 == current)[0][0]
        idx2 = np.where(parent2 == current)[0][0]
        next1 = parent1[(idx1 + 1) % n_cities]
        next2 = parent2[(idx2 + 1) % n_cities]

        candidates = []
        if next1 in remaining:
            candidates.append(next1)
        if next2 in remaining and next2 != next1:
            candidates.append(next2)

        # pick the candidate minimizing current cost
        # if none available, pick the nearest among all remaining
        if candidates:
            next_city = min(candidates, key=lambda c: dist_matrix[current, c])
        else:
            next_city = min(remaining, key=lambda c: dist_matrix[current, c])

        child.append(next_city)
        remaining.remove(next_city)

    return np.array(child)

### 3. Evolutionary Cycle

**Function:** `asymmetric_TSP(...)`

The evolutionary loop integrates all components into a GA framework:

1. **Initialization:** Random generation of diverse tours.  
2. **Evaluation:** Compute fitness for each tour using `tour_length`.  
3. **Selection:** Tournament + elitism to preserve best individuals.  
4. **Crossover:** With probability `crossover_rate`, apply **SCX** to generate new offspring.  
5. **Mutation:** With probability `mutation_rate`, apply swap mutation.  
6. **Replacement:** Form the new population and repeat for `generations` iterations.  

In [11]:
def asymmetric_TSP(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3
) -> tuple[np.ndarray, float]:
    """
    EA for ATSP with SCX crossover.
    with random initialization and operators:
      - tournament selection
      - SCX crossover
      - swap mutation
      - 2-opt local search
      - elitism
    """

    population = init_population_random(pop_size, dist_matrix)
    fitnesses = [fitness(ind, dist_matrix) for ind in population]

    best_overall = None
    best_overall_fit = float("inf")

    for gen in range(1, generations + 1):
        # sort population by fitness minimization
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        # track best global
        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        # elitism
        new_population = []
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # generation of offsprings
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)

            while np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            # SCX crossover
            if random.random() < crossover_rate:
                child1 = scx_crossover(parent1, parent2, dist_matrix)
                child2 = scx_crossover(parent2, parent1, dist_matrix)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # mutation
            if random.random() < mutation_rate:
                child1 = swap_mutation(child1)
            if random.random() < mutation_rate:
                child2 = swap_mutation(child2)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        # updation
        population = new_population.copy()
        fitnesses = [fitness(ind, dist_matrix) for ind in population]

        # progress
        if gen % 100 == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit


## Why These Components Work Well Together

The new design addresses the main challenges of the **non-metric ATSP**:

1. **SCX provides cost-driven recombination**, allowing the GA to directly exploit asymmetric costs during offspring construction.  
2. **Random initialization** ensures coverage of many diverse starting tours without bias from geometric assumptions.  
3. **Swap mutation** maintains stochastic variability while preserving feasibility.  
4. **Tournament selection with elitism** preserves the best individuals and guarantees progress despite the absence of deterministic local search.

## Experiments

### R1 10

In [12]:
# Load data
dist_matrix = np.load('/problem_r1_10.npy')

In [13]:
POP_SIZE = 100
GENERATIONS = 500
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 3


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 229.4496
Gen  100 | Best: 184.2734
Gen  200 | Best: 184.2734
Gen  300 | Best: 184.2734
Gen  400 | Best: 184.2734
Gen  500 | Best: 184.2734
Best cost: 184.27344079993895
Best tour: [6 2 9 7 1 8 0 4 5 3]


### R1 20

In [14]:
# Load data
dist_matrix = np.load('/problem_r1_20.npy')

In [15]:
POP_SIZE = 500
GENERATIONS = 1000
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.4
ELITISM = 2
TOURN_K = 4


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 650.3402
Gen  100 | Best: 344.0826
Gen  200 | Best: 344.0826
Gen  300 | Best: 337.2949
Gen  400 | Best: 337.2949
Gen  500 | Best: 337.2949
Gen  600 | Best: 337.2949
Gen  700 | Best: 337.2949
Gen  800 | Best: 337.2949
Gen  900 | Best: 337.2949
Gen 1000 | Best: 337.2949
Best cost: 337.2948721472555
Best tour: [ 7 17 15 13  8  9 16 12  4  1  3 19  2  0 14 18  5 10 11  6]


### R1 50

In [16]:
# Load data
dist_matrix = np.load('/problem_r1_50.npy')

In [17]:
POP_SIZE = 200
GENERATIONS = 1000
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.4
ELITISM = 2
TOURN_K = 3


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 2022.5170
Gen  100 | Best: 549.4352
Gen  200 | Best: 547.9743
Gen  300 | Best: 547.9743
Gen  400 | Best: 547.9743
Gen  500 | Best: 547.9743
Gen  600 | Best: 547.9743
Gen  700 | Best: 547.9743
Gen  800 | Best: 547.9743
Gen  900 | Best: 547.9743
Gen 1000 | Best: 547.9743
Best cost: 547.9743491449584
Best tour: [38 35 41 40 17 12 13 16 15 42 34 23 14  2 24 37 10 33 11 44 39 22 31 36
 29  5  0 27  3  7 49 46 21 26  8  1 20 30  6  9 45 25  4 43 32 48 18 19
 47 28]


### R1 100

In [18]:
# Load data
dist_matrix = np.load('/problem_r1_100.npy')

In [19]:
POP_SIZE = 200
GENERATIONS = 1000
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.4
ELITISM = 2
TOURN_K = 3


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 4501.9378
Gen  100 | Best: 755.7869
Gen  200 | Best: 755.2402
Gen  300 | Best: 743.5813
Gen  400 | Best: 743.5813
Gen  500 | Best: 743.5813
Gen  600 | Best: 743.5813
Gen  700 | Best: 743.5813
Gen  800 | Best: 743.5813
Gen  900 | Best: 743.5813
Gen 1000 | Best: 743.5813
Best cost: 743.581345667326
Best tour: [59 61 92 76  1  0 97 42 91 82 72 32 16 98 54 38 49 75 13 71 85 53  5 36
 58 12  4 22 80 94 51 17 43 41 78 25 56 70 84 47 23 19 45 90 48 24 57 87
 83 20 65 55 77 46 26 69 93 74 95 29 52  8 99 73  2 11 50 81 35 40 60 37
 30 86 88 44 89  7 27 21 34 62 28 68 10 15 79 96 14 64 18 31  9 63 33 39
 66  3  6 67]


### R1 200

In [20]:
# Load data
dist_matrix = np.load('/problem_r1_200.npy')

In [22]:
POP_SIZE = 300
GENERATIONS = 500
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.4
ELITISM = 2
TOURN_K = 3


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 9134.0462
Gen  100 | Best: 1087.6481
Gen  200 | Best: 1087.6481
Gen  300 | Best: 1087.6481
Gen  400 | Best: 1086.7460
Gen  500 | Best: 1052.7275
Best cost: 1052.7274970408012
Best tour: [119 180 134 157  28  23 100  52  86 128 114 109 133 182  69  32 102 160
 148 106   3 158  89  22 105  94  29  45 120 132 126 143   6 129 161  57
  10 153  20  64  58 131 162   5  73   4 199  66  65  25 156  36  67 195
  16   9 130 189 110  13  99  50  30  97  74 141 144  79  53 124 147 135
 173 123 125 150 138 191  60 168  88  85  44 181  26  40 178  84 145 159
   1 122 196 175  56 136  39 155  18 164  80  35  11 171  91 111 194  96
  98  76  59 146 140 174  37 192 139  33 112  61  87  31 115 107  63 137
 113  14 167 149 142 184  68  92  24 185 117 127  70 197  72 152  17  62
 104  54 193 166 170   7 165 188  95  19  38 151  82   8  15  90 118  48
   2 177  83  42 172 186 154  12 198  41 183 179 187  78 101 103  46  71
  75  77 169  55  47  49  51  43 163 121  21 190 176 108   0  34  2

### R1 500

In [23]:
# Load data
dist_matrix = np.load('/problem_r1_500.npy')

In [25]:
POP_SIZE = 100
GENERATIONS = 500
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.4
ELITISM = 2
TOURN_K = 5


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 24348.6871
Gen  100 | Best: 1785.3329
Gen  200 | Best: 1785.3329
Gen  300 | Best: 1785.3329
Gen  400 | Best: 1785.3329
Gen  500 | Best: 1785.3329
Best cost: 1785.3329374029427
Best tour: [332 180 159 110 194  86 182  25 126 189 203 235 185 138 247 214 373 415
   0 346  29 307  38  49 344  55 243  91 499 166 497 432 303 351 288 319
 175 481 422 476 435 433 120 350 177 234  57  28  87 405 290 200 275 136
  41 393  67 467  36 421 131 376 282 334 367  84 222 285  99 484 419 207
 139 208  23 434 238 209 121 366 123  92 116 353 330 363 443 171 295  89
 451 474 176 258  44 192 134  21 354 237 148 379 374  60 424 174 188 132
 335 250  46 244 414 239  26 162 370 482 448 438 396 236 278 227  45 127
 135 456 101  58 256 457  81 318 271 160 268  30 260 485  97   4 347 107
 369 248  12 130 466 386 297 287 314 387 149 230 473 342 312 158 199 272
 300  64  83 202 240  11 225 470 391 196 161 427 262 472  75 156   1 249
 423 286 264 231 184 251 402  96 108 284 241 302 229 220 219  77 4

### R1 1000

In [26]:
# Load data
dist_matrix = np.load('/problem_r1_1000.npy')

In [27]:
POP_SIZE = 50
GENERATIONS = 500
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.4
ELITISM = 2
TOURN_K = 3


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 49834.2561
Gen  100 | Best: 2613.8232
Gen  200 | Best: 2613.8232
Gen  300 | Best: 2613.8232
Gen  400 | Best: 2613.8232
Gen  500 | Best: 2613.8232
Best cost: 2613.823229290603
Best tour: [  4 828 134 656 973 612 208 803 505 839 961 128 589  99 402 170 276 289
 594 899 212 833  30 271 601 590 998 195 781 193 105 216 371 647 158 360
 393 143 644 815 516 707 681  31 415 207  87 506 264 142 299 519 540   9
 560 307  89 636 182 144 311 817 160 209 336 870 215 422 704 770 349 568
 386 492 774  47 335 478 880 752 885 186 825  26 867 930 140 780  14 514
 986 608 407 443 734 456 489 945 931 737 376 331 708 337 546 181  33 585
 448 383   3 993 785 442 396 227 421 795 366 769 648 964 545 270 990 343
 496 538 498 686 436 908 751 916 966  46 848 298 461 285  97 566 701 347
 553 985 802 783 153 860  24 246 494  35 814 862  75 275 610 600 375 620
 154 872 518 969 705  18  37 918 675  73 883 497 152 164 902 159 845 377
 579 703 290 328 470 868 812 960 129  49 124 175 116 373 641 370 11

# Non-metric ATSP, Negative Edge Costs and Nonzero Diagonal

These "problem_r2_N.npy" instances represent **Asymmetric Traveling Salesman Problems (ATSP)** on a **complete directed graph**.  
Unlike the metric case, it includes **negative edge costs**, a **nonzero diagonal**, and **violates the triangle inequality**.

* *Negative edges* - Some routes have negative costs, meaning certain transitions reduce the total cost instead of increasing it.  
* *Nonzero diagonal* - Staying in the same city has a nonzero cost.
* *Asymmetry* - The travel cost from A -> B differs from the cost from B -> A, making the graph **directed** and destroying the reversibility property of symmetric TSPs.  
* *Triangle inequality violated* - Direct routes are not guaranteed to be cheaper than indirect ones.

---

## Theoretical Context

Compared to the standard Asymmetric Traveling Salesman Problem (ATSP), this instance introduces two additional complexities:  
the presence of **negative arc costs** and a **nonzero diagonal**.

1. **Negative Costs and Search Landscape**    
   Allowing **negative arcs** fundamentally changes the optimization landscape: a route containing a strongly negative edge may appear beneficial locally even if it leads to a globally worse tour.  
   This introduces **deceptive local minima** and increases the ruggedness of the fitness surface, making purely local improvement heuristics less reliable.

2. **Algorithmic Implications**   
   Strategies like “shortest edge” or “nearest neighbor” may fail dramatically in the presence of negative arcs.  
   As a result, **operators that preserve direction** (like SCX) become essential, while heuristics dependent on the metric (e.g., 2-OPT, NN) lose validity.

---

## Problem Solution

To solve this problem, we applied the same algorithm previously used for solving ATSP r1 instances.




## Experiments

### R2 10

In [30]:
# Load data
dist_matrix = np.load('/problem_r2_10.npy')

In [33]:
POP_SIZE = 100
GENERATIONS = 500
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 3

best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: -251.1209
Gen  100 | Best: -411.7017
Gen  200 | Best: -411.7017
Gen  300 | Best: -411.7017
Gen  400 | Best: -411.7017
Gen  500 | Best: -411.7017
Best cost: -411.7017155524985
Best tour: [6 3 4 8 1 9 5 0 2 7]


### R2 20

In [39]:
# Load data
dist_matrix = np.load('/problem_r2_20.npy')

In [41]:
POP_SIZE = 100
GENERATIONS = 500
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 3


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: -303.1220
Gen  100 | Best: -853.4456
Gen  200 | Best: -853.4456
Gen  300 | Best: -853.4456
Gen  400 | Best: -853.4456
Gen  500 | Best: -853.4456
Best cost: -853.445592450789
Best tour: [ 6 18  8  0  5 10 14  1  3  7 11 17 13 12 19  4  9 15 16  2]


### R2 50

In [42]:
# Load data
dist_matrix = np.load('/problem_r2_50.npy')

In [44]:
POP_SIZE = 150
GENERATIONS = 750
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 5


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: -618.9883
Gen  100 | Best: -2291.4408
Gen  200 | Best: -2291.4408
Gen  300 | Best: -2291.4408
Gen  400 | Best: -2291.4408
Gen  500 | Best: -2291.4408
Gen  600 | Best: -2291.4408
Gen  700 | Best: -2291.4408
Best cost: -2291.4407511235677
Best tour: [29  9 15 21 24 36 49 30  3 47 32 33 34 14  6 25 48 31 16 13 27 12  0 39
 43 22  2 35 10 17 18 19 20  5 41 42 11 37 45 44  8 40  4 23 26  1 28 38
  7 46]


### R2 100

In [45]:
# Load data
dist_matrix = np.load('/problem_r2_100.npy')

In [46]:
POP_SIZE = 150
GENERATIONS = 750
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 5


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: -745.6262
Gen  100 | Best: -4745.5680
Gen  200 | Best: -4745.5680
Gen  300 | Best: -4745.5680
Gen  400 | Best: -4745.5680
Gen  500 | Best: -4745.5680
Gen  600 | Best: -4745.5680
Gen  700 | Best: -4745.5680
Best cost: -4745.568010531748
Best tour: [53 89 80 42 86 61 29  1  0 63 56 22  9 75 92 50 87 26 77 76 36 17 19 67
 55 47 66 71 85 23  6 33 99  7 12 37 11 62 94 73 64 49 83 34 38 54 30 90
 18 65  4 46 69 24 31  2 14 10 40  3 52 81 60 93 68 88 72 98 70 91 74 20
  5 57 39 35  8 97 25 21 28 51 79 13 82 95 41 32 15 84 59 45 16 58 78 48
 44 96 43 27]


### R2 200

In [47]:
# Load data
dist_matrix = np.load('/problem_r2_200.npy')

In [49]:
POP_SIZE = 150
GENERATIONS = 750
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.9
ELITISM = 2
TOURN_K = 5


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: -1254.7358
Gen  100 | Best: -9658.6869
Gen  200 | Best: -9658.6869
Gen  300 | Best: -9658.6869
Gen  400 | Best: -9658.6869
Gen  500 | Best: -9658.6869
Gen  600 | Best: -9663.4000
Gen  700 | Best: -9663.4000
Best cost: -9663.399998446193
Best tour: [134  20 167  46 183  55 118  61  44 148 104  66  84 169  36 130 190 128
 199 126 171  19  31 198 103 116 142 117  74  21 144  93  86  63  81  45
  94 191 177  60   6 155  25  49   8  85  64  80   3  87  43  42 149 153
 165  32 109  92  90  97  56  40 138  26 166  12 195  39 150  59  29 120
  41  96 164 185  51 175 121  24 189 136 163   4 146 143  11  13 186 114
  50   7 139  47 119 115  62  30   5   9 137  16  18 140 132 178  23 160
  28  27 170  67  14 180 156 111 168 184  91  52 179  73  83  76  75 158
  89  77 161  38  98  34 129 131 174 188  79 187  99  72 124 106 159   1
  95  70 141 182 100  22 147 112 154 197 173  88 152   0  78 162  37 110
  69 105  15  65 122 192  68 123  10  17 157 133  57 151 125  54 172  53
 145 

### R2 500

In [50]:
# Load data
dist_matrix = np.load('/problem_r2_500.npy')

In [51]:
POP_SIZE = 150
GENERATIONS = 500
CROSSOVER_RATE = 1
MUTATION_RATE = 0.9
ELITISM = 2
TOURN_K = 5


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: -2029.8967
Gen  100 | Best: -24604.9333
Gen  200 | Best: -24625.5324
Gen  300 | Best: -24625.5324
Gen  400 | Best: -24625.5324
Gen  500 | Best: -24626.9425
Best cost: -24626.9424919492
Best tour: [  3  32 492 211 130 460 353  55 454 150 446 237   4  27 156 141  12 481
 494 337 272 151 194 169  92  16 456 410 335 391  93 292  83 470 358 428
 273 471 331 448  90 204 166 264 447 409 452 349 202  10  18 416 356 249
 225 303 261 163 379 445 174 133 397 395 449 493 122   1 173 222 390 281
 495 434  45 111 104 266 406 114 451 302 146 499 342  53 366 198 296 386
  63 344 170 414 250 465 330 213 480  54 220 415 373 322 388 255 236 219
 232  43  56 117 355  62 297  72 186   2 125  11  79  46   9 431  76 142
 134 435  71 258 190 136   7  41 191 124 295 145 463 189 119  25 310 490
 277 270 265 363 328 461 269 228 207  19 147 357 407 381 309 214 317 459
 129 354 233 491 360 271 285  74 343 341 184 103 154 160  37 484 127   6
 159 293 218 221 179 316  51  31 168 256 393 346  48 352 

### R2 1000

In [52]:
# Load data
dist_matrix = np.load('/problem_r2_1000.npy')

In [53]:
POP_SIZE = 100
GENERATIONS = 500
CROSSOVER_RATE = 1
MUTATION_RATE = 0.4
ELITISM = 2
TOURN_K = 3


best_tour, best_cost = asymmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: -1755.9015
Gen  100 | Best: -49452.5685
Gen  200 | Best: -49473.3550
Gen  300 | Best: -49473.3550
Gen  400 | Best: -49473.3550
Gen  500 | Best: -49473.3550
Best cost: -49473.354952515314
Best tour: [531 934 172 619 159 984 547 644 550 477  26 101 601 595 506 743 695 721
 568  37 744 385 250 339 200 709 514 245 276 752 682 926 119 812 329 602
 979 917 739  97 770 162 725 981 935 549 193 275 109 704 149 538 472 479
 174 900 226 185 196 424 767 168 971 591  53 669 166 747 940 186 604 840
 165 621 618 871 978  71 445 396 698 355 878 716   9 503 518 285  68 593
 751 368  67 982 582 290 596 296  85  95 990 736 985   8 811 748 844 654
 532 629 723 610 462 671 948 520 620   5 886  44 715 324 521 719 864 104
 785 282 711 761 835 151 986  16 380 659  10 239 758 252 939 737  59 123
 865 450 882 349 836 311 542  19 627 305 207 420 754 839 857 397 344 100
  64 973   4 953  32 284 510 827  74 569 342 947 113  33 350 662 126 606
 898 657 176 163 423 288 802 819 989 880 428 141 182 68

# Test problem

In [55]:
# Load data
dist_matrix = np.load('/test_problem.npy')

In [56]:
from itertools import combinations

# Tests
print("Contains negative values?", np.any(dist_matrix < 0))
print("Zero diagonal?", np.allclose(np.diag(dist_matrix), 0.0))
print("Symmetric matrix?", np.allclose(dist_matrix, dist_matrix.T))
triangle_ok = all(
    dist_matrix[x, y] <= dist_matrix[x, z] + dist_matrix[z, y]
    for x, y, z in combinations(range(dist_matrix.shape[0]), 3)
)
print("Satisfies the triangle inequality?", triangle_ok)
print("Number of cities:", len(dist_matrix))

Contains negative values? False
Zero diagonal? True
Symmetric matrix? True
Satisfies the triangle inequality? True
Number of cities: 20


In [61]:
POP_SIZE = 10
GENERATIONS = 100
CROSSOVER_RATE = 0.95
MUTATION_RATE = 0.3
ELITISM = 2
TOURN_K = 2
TWO_OPT_RATE = 1

best_tour, best_cost = symmetric_TSP(
    dist_matrix=dist_matrix,
    pop_size=POP_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    elitism_count=ELITISM,
    tournament_k=TOURN_K,
    two_opt_rate=TWO_OPT_RATE
)

print("Best cost:", best_cost)
print("Best tour:", best_tour)

Gen    1 | Best: 3309.3700
Gen   10 | Best: 2823.7900
Gen   20 | Best: 2823.7900
Gen   30 | Best: 2823.7900
Gen   40 | Best: 2823.7900
Gen   50 | Best: 2823.7900
Gen   60 | Best: 2823.7900
Gen   70 | Best: 2823.7900
Gen   80 | Best: 2823.7900
Gen   90 | Best: 2823.7900
Gen  100 | Best: 2823.7900
Best cost: 2823.7899999999995
Best tour: [ 7 17  6 19 18  5  3  1 16 11 13 10 14  8 15 12  9  4  2  0]
